In [1]:
from pydantic_ai.settings import ModelSettings

from aurelian.agents.gocam import GOCAMDependencies
from aurelian.agents.gocam.gocam_agent import gocam_reviewer_agent, gocam_review_summarizer_agent

deps = GOCAMDependencies() 

/Users/cjm/Library/Caches/pypoetry/virtualenvs/aurelian-JldJeJhF-py3.11/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
import os
cborg_api_key = os.environ.get("CBORG_CONTEXTUALIZER_API_KEY")

In [3]:
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIModel


# not to be confused with GO-CAM Model
ai_model = OpenAIModel(
    "anthropic/claude-sonnet",
    provider=OpenAIProvider(
        base_url="https://api.cborg.lbl.gov",
        api_key=cborg_api_key),
)

settings = ModelSettings(
    max_tokens=64000,
)


In [4]:
collection = deps.collection

['gocams']


In [5]:
qr = collection.find({}, limit=-1) 

In [6]:
len(qr.rows)

944

In [7]:
from gocam.datamodel import Model


def is_causal(m: dict):
    model = Model(**m)
    causal_associations = []
    for a in model.activities or []:
        causal_associations.extend(a.causal_associations or [])
    return len(causal_associations) > 2

models = [m for m in qr.rows if is_causal(m)]
len(models)

740

In [8]:
!mkdir -p go-cam-reviews

In [9]:
import nest_asyncio
nest_asyncio.apply()


In [10]:
import os
import asyncio

for m in models:
    model = Model(**m)
    title = model.title
    # make safe for file name; replace ALL non-alphanumeric characters with "_"
    title = "".join(c if c.isalnum() else "_" for c in title)
    model_num = model.id.split(":")[-1]
    fn = f"go-cam-reviews/{model_num}-{title}.md"
    # if file exists, skip
    if os.path.exists(fn):
        print(f"File {fn} already exists, skipping.")
        continue
        
    command = f"Review {model.id}"
    print(f"COMMAND: {command}")
    #result = gocam_reviewer_agent.run_sync(command)
    
    #async def my_async_function():
    #    r = await gocam_reviewer_agent.run(command,  deps=deps)
     #   return r.data
        
    try:
        result = gocam_reviewer_agent.run_sync(command, deps=deps, model=ai_model, model_settings=settings)
    except Exception as e:
        print(f"Error: {e}")
        continue
    print(result.data)
    
    with open(fn, "w") as f:
        f.write(result.data)

File go-cam-reviews/56170d5200000012-kctd10_in_heart_development_PMID_24430697_.md already exists, skipping.
File go-cam-reviews/568b0f9600000284-Antibacterial_innate_immune_response_in_the_intestine_via_MAPK_cascade__C__elegans_.md already exists, skipping.
File go-cam-reviews/5745387b00001516-Canonical_glycolysis_1__Mouse_.md already exists, skipping.
File go-cam-reviews/57c82fad00000695-F13a1_and_F13b_mouse_blood_clotting.md already exists, skipping.
File go-cam-reviews/5966411600000170-Intestinal_smooth_muscle_cell_differentiation_via_foxo1a__mir145__tgfr1b__ltbp3__zeb1a__D__rerio_.md already exists, skipping.
File go-cam-reviews/59cc772c00000000-IL1beta_mediated_signaling_pathway__Human_.md already exists, skipping.
File go-cam-reviews/59dc728000000555-_Wnt_FGF_Notch_signaling_inregulation_of_HSC_specification_via_dlc__dld__fgfr4__wnt16__D__rerio_.md already exists, skipping.
File go-cam-reviews/5ae3b0f600000435-Regulation_of_sprouting_angiogenesis_via_dll4__egfl6__itgb1a__D__reri

In [11]:
print("done")

done


In [12]:
# find all reviews in go-cam-reviews/ by glob searching *md
import glob
files = glob.glob("go-cam-reviews/*.md")

In [20]:
#cborg_api_key

In [18]:
summarization_model = OpenAIModel(
    "anthropic/claude-sonnet",
    provider=OpenAIProvider(
        base_url="https://api.cborg.lbl.gov",
        api_key=cborg_api_key),
)

In [ ]:
for fn in files:
    print(fn)
    content = open(fn).read()
    #print(len(content))
    # content = content[:200]
    command = f"Summarize this review: {content}"
    #result = gocam_reviewer_agent.run_sync(command, deps=deps, model=ai_model, model_settings=settings)
    result = gocam_review_summarizer_agent.run_sync(command, deps=deps, model=summarization_model)
    #print(result.data)
    # save the json in the same dir
    with open(fn + ".json", "w") as f:
        f.write(result.data.model_dump_json(indent=2))
    #break


go-cam-reviews/63f809ec00000560-Non_canonical_activity_of_the_RQC_complex_via_CAT_tailing_by_NEMF_that_promotes_degradation_of_incompletely_synthesized_nascent_proteins_by_the_DesCEND_pathway__Human_.md
go-cam-reviews/62900b6400002879-Dermatan_sulfate_catabolic_process__Mouse_.md
go-cam-reviews/66187e4700003043-GDP_mannose_biosynthetic_process_from_glucose__Mouse_.md
go-cam-reviews/66b5638000002630-GRIM1_activates_GRID1_via_GNAQ_in_dopamine_neurons__Human__.md
go-cam-reviews/56170d5200000012-kctd10_in_heart_development_PMID_24430697_.md
go-cam-reviews/62b4ffe300001832-Activation_of_CASP7_by_CASP1_in_response_to_lipopolysaccharide__leading_to_cleavage_and_degradation_of_PARP1_by_CASP7__Human_.md
go-cam-reviews/62b4ffe300002814-Poxvirus_OPG204_inhibition_of_type_I_interferon_signaling___Human_Poxvirus__.md
go-cam-reviews/65c57c3400000100-Resolution_of_protein_DNA_cross_links_by_HMCES__BRIP1_FANCJ_and_SPRTN__Human__.md
go-cam-reviews/665912ed00000459-dolichol_linked_oligosaccharide_biosyn

In [ ]:
print("Done")